In [13]:
class BitSet:
    def __init__(self, elementNum):
        self.bytesNum = round(elementNum/8)
        self.set = bytearray(self.bytesNum)
        
    def insert(self, index):
        if(index >= 0 and index>>3 < self.bytesNum):
            self.set[index >> 3] = self.set[index >> 3] | (1 << int(index & 7)) 
            return True;
        return False;
    
    def isElement(self, index):
        if(index >= 0 and index>>3 < self.bytesNum and (self.set[index >> 3] & (1<<(index & 7)))):
            return True
        return False
        
    def AND(self, bs):
        a = BitSet(self.bytesNum*8)
        for i in range(self.bytesNum):
            a.set[i] = self.set[i] & bs.set[i]
        return a


In [11]:
def processBar(index, totalNum):
    bar = '#'*int((index/totalNum)*50)
    sys.stdout.write(str(int((index/totalNum)*100))+'%  ||'+bar+'->'+str(index)+'/'+str(totalNum)+"\r")
    #sys.stdout.flush()

In [12]:
def is_ch(uchar):
        """判断一个unicode是否是汉字"""
        if uchar >= u'\u4e00' and uchar<=u'\u9fa5':
                return True
        else:
                return False

In [45]:
word2int = {}
idx = 0
with open('dict/dict',encoding='utf8') as fin:
    for line in fin.readlines():
        word = line.split(' ')[0]
        if(word not in word2int):
            word2int[word] = idx
            idx += 1

wordsNum = len(word2int)
print('size of dict: ',len(word2int))

size of dict:  583280


In [41]:
import jieba
jieba.load_userdict('dict/dict')

In [15]:
def readIIDX(file):
    with open(file,encoding='utf8') as f:
        s = f.read()[:-1].split(' ')
    bSet = BitSet(len(s)*8)
    for i in range(bSet.bytesNum):
        bSet.set[i] = int(s[i])
    return bSet

In [55]:
pagesNum = 3029940 # 46*256*256+59*256+180 最后一个文件 pages/46/59/180

partsNum = 47 # 分别位于pages/[0-46]

def pageid2fpath(partID,id):
    fpath = 'pages/'+str(partID)+'/'+\
            str(id>>8 & 255)+'/'+str(id&255)
    return fpath

def chid2fpath(partID,id): #partID = [0-46]
    fpath = 'iidx/'+str(partID)+'/'+\
            str(id>>16 & 255)+'/'+str(id>>8 & 255)+'/'+str(id&255)+'.iidx'
    return fpath 

In [58]:
print(pageid2fpath(8,18945))
print(len(readIIDX(chid2fpath(5,word2int['没有'])).set))

pages/8/74/1
8192


In [59]:
def mask(bytes):
    r = BitSet(bytes*8)
    for i in range(bytes):
        r.set[i] = 255
    return r

bsMask = mask(round(256*256/8)) # 掩码

In [92]:
stopWords = ['上','的','什么','什麼','是','大','第几',\
             '过','多少','多少次','？','哪','谁','哪里',\
            '哪个','哪种','哪位','哪座','指','和']

def findDoc(query):
    keyList = []
    targets = [bsMask for i in range(partsNum)] #对每个部分读取倒排索引
    for word in jieba.cut_for_search(query):
        if(word not in stopWords):
            for i in range(partsNum):
                try:
                    #print(word)
                    keyList.append(word2int[word])
                    targets[i] = targets[i].AND(readIIDX(chid2fpath(i,word2int[word])))
                except KeyError as e:
                    pass
    result = []
    for i in range(partsNum):
        for j in range(pagesNum):
            if(targets[i].isElement(j)):
                result.append(pageid2fpath(i,j))  
    #print('doc number: ',len(result))
    return result

In [93]:
from langconv import *

line = '為当今世界著名的[[美國]][[網球]]女子巨星，網球史上最伟大的女子选手之一，\
    美國體育史上最佳女子網球选手（與[[克里斯·埃弗特|Chris Evert]]並列），\
    全世界第一位網球大滿貫黑人女單冠軍，單打最高世界排名第一，\
    23座大滿貫女單冠軍得主，網球史上第二位女子單打生涯全滿貫得主。'
# 转换繁体到简体
line = Converter('zh-hans').convert(line)
print(line)
# 转换简体到繁体
line = Converter('zh-hant').convert(line)
print(line)

为当今世界著名的[[美国]][[网球]]女子巨星，网球史上最伟大的女子选手之一，    美国体育史上最佳女子网球选手（与[[克里斯·埃弗特|Chris Evert]]并列），    全世界第一位网球大满贯黑人女单冠军，单打最高世界排名第一，    23座大满贯女单冠军得主，网球史上第二位女子单打生涯全满贯得主。
為當今世界著名的[[美國]][[網球]]女子巨星，網球史上最偉大的女子選手之一，    美國體育史上最佳女子網球選手（與[[克裡斯·埃弗特|Chris Evert]]併列），    全世界第一位網球大滿貫黑人女單冠軍，單打最高世界排名第一，    23座大滿貫女單冠軍得主，網球史上第二位女子單打生涯全滿貫得主。


In [94]:
def findLine(query, minLen=10, maxReturn=1):
    s = []
    docList = findDoc(query)[:10]
    keywords = [word for word in jieba.cut_for_search(query)]
    #keywords = '.'.join(jieba.cut(query,cut_all=False)).split('.')


    for doc in docList:
        with open(doc,encoding='utf8') as f:
            for line in f.readlines():
                if(len(line) > minLen):
                    tf = {}
                    for word in jieba.cut_for_search(line):
                        if(word in keywords and word not in stopWords):
                            tf[word] = 1 
                    bingo = len(tf)
                    s.append(('位于 '+doc+'   '+line,bingo))
                    
    query = Converter('zh-hant').convert(query)
    docList = findDoc(query)[:10]
    keywords = [word for word in jieba.cut_for_search(query)]
    #print(keywords)

    for doc in docList:
        with open(doc,encoding='utf8') as f:
            for line in f.readlines():
                if(len(line) > minLen):
                    tf = {}
                    for word in jieba.cut_for_search(line):
                        if(word in keywords and word not in stopWords):
                            tf[word] = 1 
                    bingo = len(tf)
                    s.append(('位于 '+doc+'   '+line,bingo))
    s = sorted(s,key=lambda x:x[1], reverse=1)
    return [item[0] for item in s[:maxReturn]]

In [95]:
def showfile(fpath):
    with open(fpath, encoding='utf8') as f:
        print(f.read())

In [96]:
#findLine('阿根廷国家足球队赢得过多少次美洲杯冠军',maxReturn=10)

In [97]:
questions = []
with open('wdm_assignment_3_samples.txt',encoding='utf8') as fin:
    for line in fin.readlines():
        questions.append(line.split('\t')[0])

questions[:10]

['马尔代夫的第一大支柱产业是什么？',
 '澳大利亚是南半球面积第几大的国家？',
 '阿根廷国家足球队赢得过多少次美洲杯冠军？',
 '平均人口密度最高的大洲是？',
 '中国历史上最杰出的浪漫主义诗人是？',
 '甲午战争爆发的标志是？',
 '《华英字典》的作者是？',
 '《资治通鉴》的撰写一共耗时多少年？',
 '世界上首位从南坡登上珠穆朗玛峰的女性的国籍是？',
 '最后宣布废除奴隶制度的行政地区是？']

In [98]:
for q in questions:
    print('问:',q)
    print('答:',findLine(q,maxReturn=1))
    print()

问: 马尔代夫的第一大支柱产业是什么？


KeyboardInterrupt: 